In [1]:
import numpy as np
import pandas as pd

def load_data(file_path):
    # Load the Excel file
    xls = pd.ExcelFile(file_path)
    # Load the "Games" sheet into a DataFrame
    games_data = pd.read_excel(xls, sheet_name='Games')
    return games_data

def process_games_data(df):
    # Add the "Gameweek" column
    df['Gameweek'] = 'Gameweek ' + (df.index + 1).astype(str)
    # Determine the winning team
    conditions = [
        df['Team A Goals'] > df['Team B Goals'],
        df['Team A Goals'] < df['Team B Goals'],
        df['Team A Goals'] == df['Team B Goals']
    ]
    choices = ['Team A', 'Team B', 'Draw']
    df['Winning Team'] = np.select(conditions, choices, default='Draw')
    # Calculate the total number of goals
    df['Total Goals'] = df['Team A Goals'] + df['Team B Goals']
    # Return the reordered DataFrame
    return df[['Date', 'Gameweek', 'Team A Goals', 'Team B Goals', 'Winning Team', 'Total Goals']]

# Load the games data and process it
games_data = load_data('FantaSPL_V2.3.xlsx')
processed_games_data = process_games_data(games_data)

# Load the "Points" sheet into a DataFrame
points_data = pd.read_excel('FantaSPL_V2.3.xlsx', sheet_name='Points')

# Count the number of players for each date
player_counts_per_date = points_data.groupby('Date').agg({'Player': 'nunique'}).reset_index()
player_counts_per_date.columns = ['Date', 'Number of Players']

# Merge the player counts with the processed_games_data dataframe based on the Date column
merged_data = pd.merge(processed_games_data, player_counts_per_date, on='Date', how='left')




In [2]:

def determine_game_type(row):
    if row['Number of Players'] <= 10:
        return '5-a-side'
    elif row['Number of Players'] <= 16:
        return '7-a-side'
    else:
        return '11-a-side'

merged_data['Game Type'] = merged_data.apply(determine_game_type, axis=1)
merged_data


,Date,Gameweek,Team A Goals,Team B Goals,Winning Team,Total Goals,Number of Players,Game Type
0,2023-01-12,Gameweek 1,18,13,Team A,31,14,7-a-side
1,2023-01-19,Gameweek 2,4,7,Team B,11,22,11-a-side
2,2023-01-26,Gameweek 3,5,8,Team B,13,16,7-a-side
3,2023-02-02,Gameweek 4,8,13,Team B,21,14,7-a-side
4,2023-02-09,Gameweek 5,6,11,Team B,17,14,7-a-side
5,2023-02-23,Gameweek 6,5,7,Team B,12,16,7-a-side
6,2023-03-02,Gameweek 7,9,7,Team A,16,14,7-a-side
7,2023-03-09,Gameweek 8,8,9,Team B,17,14,7-a-side
8,2023-03-16,Gameweek 9,8,14,Team B,22,14,7-a-side
9,2023-03-23,Gameweek 10,5,5,Draw,10,14,7-a-side


In [3]:

# Load the "Parameters" sheet into a DataFrame
parameters_data = pd.read_excel('FantaSPL_V2.3.xlsx', sheet_name='Parameters')

# Extract participation points for each game type based on the new structure
participation_points = {
    '5-a-side': parameters_data.loc[parameters_data['Parameter'] == 'Participation', '5-a-side'].values[0],
    '7-a-side': parameters_data.loc[parameters_data['Parameter'] == 'Participation', '7-a-side'].values[0],
    '11-a-side': parameters_data.loc[parameters_data['Parameter'] == 'Participation', '11-a-side'].values[0]
}

# Creating a new "Points" table from the "Points" sheet
new_points_data = points_data.copy()

# Merging with merged_data to get the game type for each player's participation
new_points_data = pd.merge(new_points_data, merged_data[['Date', 'Game Type', 'Team A Goals', 'Team B Goals', 'Winning Team', 'Total Goals']], on='Date', how='left')

# Adding the "Participation Points" column based on the game type
new_points_data['Participation Points'] = new_points_data['Game Type'].map(participation_points)

In [4]:

# Extracting points for each game outcome based on game type from the "Parameters" sheet
win_points = {
    '5-a-side': parameters_data.loc[parameters_data['Parameter'] == 'Win', '5-a-side'].values[0],
    '7-a-side': parameters_data.loc[parameters_data['Parameter'] == 'Win', '7-a-side'].values[0],
    '11-a-side': parameters_data.loc[parameters_data['Parameter'] == 'Win', '11-a-side'].values[0]
}

draw_points = {
    '5-a-side': parameters_data.loc[parameters_data['Parameter'] == 'Draw', '5-a-side'].values[0],
    '7-a-side': parameters_data.loc[parameters_data['Parameter'] == 'Draw', '7-a-side'].values[0],
    '11-a-side': parameters_data.loc[parameters_data['Parameter'] == 'Draw', '11-a-side'].values[0]
}

# Assuming points for loss is 0 for all game types
loss_points = {'5-a-side': 0, '7-a-side': 0, '11-a-side': 0}

def compute_game_outcome_points(row):
    # If the game was a draw
    if row['Winning Team'] == 'Draw':
        return draw_points[row['Game Type']]
    # If the player's team won
    elif row['Winning Team'] == row['Team']:
        return win_points[row['Game Type']]
    # If the player's team lost
    else:
        return loss_points[row['Game Type']]

# Computing the "Game Outcome Points" for each player using the updated table
new_points_data['Game Outcome Points'] = new_points_data.apply(compute_game_outcome_points, axis=1)

In [5]:
# Extracting points for goals scored based on game type from the "Parameters" sheet
goal_points = {
    '5-a-side': parameters_data.loc[parameters_data['Parameter'] == 'Goal', '5-a-side'].values[0],
    '7-a-side': parameters_data.loc[parameters_data['Parameter'] == 'Goal', '7-a-side'].values[0],
    '11-a-side': parameters_data.loc[parameters_data['Parameter'] == 'Goal', '11-a-side'].values[0]
}
# Computing the "Goal Points" for each player
new_points_data['Goal Points'] = new_points_data['Goals'] * new_points_data['Game Type'].map(goal_points)

In [6]:
# Extracting points for own goals scored based on game type from the "Parameters" sheet
own_goal_points = {
    '5-a-side': parameters_data.loc[parameters_data['Parameter'] == 'Own Goal', '5-a-side'].values[0],
    '7-a-side': parameters_data.loc[parameters_data['Parameter'] == 'Own Goal', '7-a-side'].values[0],
    '11-a-side': parameters_data.loc[parameters_data['Parameter'] == 'Own Goal', '11-a-side'].values[0]
}

# Computing the "Own Goal Points" for each player
new_points_data['Own Goal Points'] = new_points_data['Own Goals'] * new_points_data['Game Type'].map(own_goal_points)

In [7]:
# Extracting points for penalties based on game type from the "Parameters" sheet
penalty_points = {
    '5-a-side': parameters_data.loc[parameters_data['Parameter'] == 'Penalty', '5-a-side'].values[0],
    '7-a-side': parameters_data.loc[parameters_data['Parameter'] == 'Penalty', '7-a-side'].values[0],
    '11-a-side': parameters_data.loc[parameters_data['Parameter'] == 'Penalty', '11-a-side'].values[0]
}
# Computing the "Penalty Points" for each player
new_points_data['Penalty Points'] = new_points_data['Penalty'] * new_points_data['Game Type'].map(penalty_points)

In [8]:
# Extracting points SPL based on game type from the "Parameters" sheet
spl_points = {
    '5-a-side': parameters_data.loc[parameters_data['Parameter'] == 'SPL Bonus', '5-a-side'].values[0],
    '7-a-side': parameters_data.loc[parameters_data['Parameter'] == 'SPL Bonus', '7-a-side'].values[0],
    '11-a-side': parameters_data.loc[parameters_data['Parameter'] == 'SPL Bonus', '11-a-side'].values[0]
}
# Computing the "SPL Points" for each player
new_points_data['SPL Bonus'] = new_points_data['SPL Bonus'] * new_points_data['Game Type'].map(spl_points)

In [9]:

# Extracting points MVP based on game type from the "Parameters" sheet
mvp_points = {
    '5-a-side': parameters_data.loc[parameters_data['Parameter'] == 'MVP', '5-a-side'].values[0],
    '7-a-side': parameters_data.loc[parameters_data['Parameter'] == 'MVP', '7-a-side'].values[0],
    '11-a-side': parameters_data.loc[parameters_data['Parameter'] == 'MVP', '11-a-side'].values[0]
}
# Computing the "MVP Points" for each player
new_points_data['MVP'] = new_points_data['MVP'] * new_points_data['Game Type'].map(mvp_points)
# Extracting points Friend Referrals based on game type from the "Parameters" sheet
friend_referral_points = {
    '5-a-side': parameters_data.loc[parameters_data['Parameter'] == 'Friend Referral', '5-a-side'].values[0],
    '7-a-side': parameters_data.loc[parameters_data['Parameter'] == 'Friend Referral', '7-a-side'].values[0],
    '11-a-side': parameters_data.loc[parameters_data['Parameter'] == 'Friend Referral', '11-a-side'].values[0]
}


In [10]:
# Computing the "Goal Points" for each player
new_points_data['Friend Referral'] = new_points_data['Friend Referrals'] * new_points_data['Game Type'].map(friend_referral_points)
# Creating a mapping dictionary for player positions based on game type
position_mapping = {
    '11-a-side': 'Position 11-a-side',
    '7-a-side': 'Position 7-a-side',
    '5-a-side': 'Position 5 a-side'
}

In [11]:
#Adding postions based on matchtype
players_data = pd.read_excel('FantaSPL_V2.3.xlsx', sheet_name='Players')

# Function to get player position based on game type
def get_player_position(row):
    game_type = row['Game Type']
    player = row['Player']
    
    # Get the player position column name based on game type
    position_col = position_mapping[game_type]
    
    # Return the player position based on game type and player name
    return players_data.loc[players_data['Player'] == player, position_col].values[0] if player in players_data['Player'].values else np.nan

# Apply the function to get the player position and add it as a new column to the new_points_data DataFrame
new_points_data['Player Position'] = new_points_data.apply(get_player_position, axis=1)
new_points_data.head()

,Date,Player,Team,Goals,Own Goals,SPL Bonus,MVP,Friend Referrals,Penalty,Game Type,...,Team B Goals,Winning Team,Total Goals,Participation Points,Game Outcome Points,Goal Points,Own Goal Points,Penalty Points,Friend Referral,Player Position
0,2023-01-12,Andrea Limonta,Team B,0,0,3,0,0,0,7-a-side,...,13,Team A,31,4,0,0,0,0,0,Defensive
1,2023-01-12,Luca Stoppi,Team B,2,1,0,0,0,0,7-a-side,...,13,Team A,31,4,0,2,-2,0,0,Defensive
2,2023-01-12,Riccardo Ricci,Team B,0,0,0,0,0,0,7-a-side,...,13,Team A,31,4,0,0,0,0,0,Offensive
3,2023-01-12,Marco Taglio,Team B,3,0,0,0,0,0,7-a-side,...,13,Team A,31,4,0,3,0,0,0,Offensive
4,2023-01-12,Mazzu,Team B,7,0,0,0,0,0,7-a-side,...,13,Team A,31,4,0,7,0,0,0,Offensive


In [ ]:


9. Defensive Score:
This score varies based on the player's position and the match type:

For 11-a-side matches:

Goalkeepers: Defense score minus the number of goals conceded plus additional goalkeeper score.
Defenders: Defense score minus the number of goals conceded.
Midfielders: Half of (Defense score minus the number of goals conceded), rounded up.

For 7-a-side matches:

Goalkeepers: Defense score minus the number of goals conceded plus additional goalkeeper score.
Defensive: Defense score minus the number of goals conceded.
For 5-a-side matches:

Goalkeepers: Defense score minus the number of goals conceded plus additional goalkeeper score.
Outfield: Defense score minus the number of goals conceded.


10. Midfield Score:
This score varies based on the player's position and the match type:

For 11-a-side matches:

Midfielders: Goal difference (if positive).
Forwards: Goal difference divided by 3, rounded up (if positive).
For 7-a-side matches:

Offensive: Goal difference (if positive).
For 5-a-side matches:

Offensive: Goal difference (if positive).
Note: The term "goal difference" refers to the difference between goals scored and goals conceded.

This comprehensive breakdown should provide clarity on how each player's points are determined based on various criteria and match types.